In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import linalg
from Mesh import Mesh

import matplotlib.pyplot as plt


In [2]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
m = Mesh('data/coil_1.2_MM.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

print('Computing Laplacian')
K, M = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = linalg.eigh(K,M)

Computing Laplacian
Computing eigen values


In [4]:
# send all relevant numpy arrays to torch tensors
K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)
X = torch.from_numpy(m.verts).to(device)

In [5]:
# in the paper we used 50 eigenvalues so set k to 50
k = 50

In [ ]:
# Build the neural network that maps coordinates -> k outputs per node
class MLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=k, hidden=[64,64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h, dtype=torch.double))
            layers.append(nn.SiLU())
            last = h
        layers.append(nn.Linear(last, out_dim, dtype=torch.double))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# Instantiate model and optimizer
model = MLP().to(device)
# Initialize all layers (Xavier), final layer small
for name, p in model.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
for p in model.net[-1].parameters():  # last Linear
    if p.ndim == 2:
        nn.init.normal_(p, std=1e-3)
    else:
        nn.init.zeros_(p)

# Helper: given A (N x k), return M-orthonormalized U = A (A^T M A)^{-1/2}
def m_orthonormalize(A, M):
    # A: (N, k), M: (N, N)
    # compute B = A^T M A (k x k)
    B = A.T @ (M @ A)  # k x k
    # symmetrize B
    B = 0.5*(B + B.T)
    # compute inverse sqrt of B via eigendecomposition (k small)
    s, Q = torch.linalg.eigh(B)  # s are eigenvalues
    # regularize small eigenvalues
    s_clamped = torch.clamp(s, min=1e-12)
    inv_sqrt = Q @ torch.diag(1.0/torch.sqrt(s_clamped)) @ Q.T
    U = A @ inv_sqrt
    return U

lr_start = 0.01
lr_end = 0.0001
max_epochs = 100_000
print_every = 1_000
loss_history = []

optimizer = optim.Adam(model.parameters(), lr=lr_start)
decay_factor = (lr_end / lr_start) ** (1 / max_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_factor)

for epoch in range(1, max_epochs+1):
    model.train()
    optimizer.zero_grad()
    A = model(X)  # N x k
    U = m_orthonormalize(A, M)  # U is M-orthonormal

    UKU = U.T @ (K @ U)
    # symmetrize for numerical safety
    UKU = 0.5*(UKU + UKU.T)
    loss = torch.trace(UKU)
    loss.backward()
    optimizer.step()
    scheduler.step()
    loss_history.append(loss.item())
    if epoch % print_every == 0 or epoch == 1:
        print(f"Epoch {epoch:4d}, loss={loss.item():.6f}")

Epoch    1, loss=195.006185
Epoch 1000, loss=271.388413
Epoch 2000, loss=261.308455


In [ ]:
np.set_printoptions(suppress=True, precision=6)

model.eval()
with torch.no_grad():
    A_final = model(X)

    U_final= m_orthonormalize(A_final, M)
    UKU = U_final.T @ (K @ U_final)

    print(f"The shape of U.t @ K @ U is: {UKU.shape}")

    approx_eigs = np.round(torch.diag(UKU).cpu().numpy(), 8)
    # For better comparison, we can compute Ritz values from subspace U by solving small generalized eigenproblem
    # (U^T K U) c = mu (U^T M U) c, but U^T M U = I so just eig of UK_final
    mu, Wsmall = np.linalg.eigh(UKU.cpu().numpy())
    mu = np.real(mu)
    # sort
    idx = np.argsort(mu)
    mu = mu[idx]
    print("\nLearned Ritz values (from U^T K U):", np.round(mu[:5], 6))
    print("Reference eigenvalues (first k):   ", np.round(eigvals[:5], 6))

In [ ]:
ax =  plt.figure(figsize=(10,10)).add_subplot(projection='3d')
surf = ax.plot_trisurf(m.verts[:,0], m.verts[:,1], m.verts[:,2], triangles = m.connectivity)
plt.show()

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
A_final = model(X)
U_final = m_orthonormalize(A_final, M)


Ksmall = U_final.T @ K @ U_final
Msmall = U_final.T @ M @ U_final
L = torch.linalg.cholesky(Msmall)
Linv = torch.linalg.inv(L)
Khat = Linv @ Ksmall @ Linv.T
mu, _ = torch.linalg.eigh(Khat)


mu = mu.cpu().numpy()
idx = np.argsort(mu)
mu = mu[idx]


print(f"The shape of U.T @ K @ U is: {U_final.T @ K @ U_final.shape}")
print("\nLearned Ritz values (from U^T K U):", np.round(mu[:5], 6))
print("Reference eigenvalues (first k): ", np.round(eigvals[:5], 6))